Unit stats are stored in a dictionary. Stats we care about for combat sim are HP(int), DPS(float), Range(bool), Armor(int), Attributes(set string), Targetable(set string), Bonuses(set string), Bonus DPS(float). We also want to track Minerals(int), Gas(int), Supply(int), and Time(int) for the linear program.

All stats assume no upgrades. Shields are added to health to get HP. Bonuses is a set of string attributes that this unit will deal bonus damage to. Bonus DPS is the damage the unit will deal to an enemy unit after accounting for bonuses. Targetable is a set of string attributes that this unit can, either ground and/or air.

Will probably include a separate exception within the simulator itself to deal with SCV healing Mechs and Medivacs healing Bio.

In [1]:
import random

In [19]:
Terran_Units = {}
Units = {}

Marine = {}
# Combat stats
Marine['hp'] = 45
Marine['dps'] = 9.8
Marine['ranged'] = True
Marine['armor'] = 0
Marine['attributes'] = {'Biological', 'Light', 'Ground'}
Marine['type'] = {'Ground'}
Marine['targetable'] = {'Ground', 'Air'}
Marine['bonuses'] = {}
Marine['bonus_dps'] = 0
Marine['healer'] = False
# Resource stats
Marine['mineral'] = 50
Marine['gas'] = 0
Marine['time'] = 18
Marine['supply'] = 1
Terran_Units['Marine'] = Marine
Units['Marine'] = Marine

SCV ={}
# Combat stats
SCV['hp'] = 45
SCV['dps'] = 4.67
SCV['ranged'] = False
SCV['armor'] = 0
SCV['attributes'] = {'Biological', 'Light', 'Mechanical', 'Ground'}
SCV['targetable'] = {'Ground'}
SCV['type'] = {'Ground'}
SCV['bonuses'] = {}
SCV['bonus_dps'] = 0
SCV['healer'] = True
# Resource stats
SCV['mineral'] = 50
SCV['gas'] = 0
SCV['time'] = 12
SCV['supply'] = 1
Terran_Units['SCV'] = SCV
Units['SCV'] = SCV

The SCV and Medivac names will be flagged in the simulator so that they repair/heal with priority. Medivacs heal a flat 12.6 hps whereas SCVs heal rate differs per unit. Repair rates for SCV found here: https://tl.net/forum/sc2-strategy/160734-repair-speed-per-unit

In [3]:
medivac_heal = 12.6

scv_repair = {}
scv_repair['SCV'] = 2.6
scv_repair['Hellion'] = 3
scv_repair['Siege'] = 3.5
scv_repair['Thor'] = 6.6
scv_repair['Viking'] = 2.9
scv_repair['Medivac'] = 3.5
scv_repair['Raven'] = 2.3
scv_repair['Banshee'] = 2.3

Function for running combat sim.
Input: dictionary of enemy army, dictionary of testing army
    dictionary of armies has unit name as keys, unit count as values
Output: dictionary of testing army

In [41]:
# test enemy army
enemy_army_comp = {}
enemy_army_comp['SCV'] = 2
enemy_army_comp['Marine'] = 3

# test test_army
test_army_comp = {}
test_army_comp['SCV'] = 0
test_army_comp['Marine'] = 3

We will keep track of individual units within our army simulation by using a custom army Unit class

In [59]:
class Unit:
    def __init__(self, name):
        """
        Creates a new Unit object with the stats of the given name
        """
        self.name = name
        self.hp = Units[name]['hp']
        self.dps = Units[name]['dps']
        self.ranged = Units[name]['ranged']
        self.armor = Units[name]['armor']
        self.attributes = Units[name]['attributes']
        self.type = Units[name]['type']
        self.targetable = Units[name]['targetable']
        self.bonuses = Units[name]['bonuses']
        self.bonus_dps = Units[name]['bonus_dps']
        self.healer = Units[name]['healer']
    
    def __str__(self):
        return self.name
    
    def __repr__(self):
        return str(self.name) + "_HP:" + str(self.hp)

In [42]:
enemy_army = build_army(enemy_army_comp)
test_army = build_army(test_army_comp)

In [43]:
enemy_army

[SCV_HP:45, SCV_HP:45, Marine_HP:45, Marine_HP:45, Marine_HP:45]

In [44]:
test_army

[Marine_HP:45, Marine_HP:45, Marine_HP:45]

In [47]:
for enemy in enemy_army:
    print(enemy)

SCV
SCV
Marine
Marine
Marine


In [ ]:
deal_damage(test_army, enemy_army)

In [ ]:
enemy_army

In [ ]:
marine1 = Unit("Marine")
marine2 = Unit("Marine")
marine3 = marine1

In [ ]:
marine1.hp = 0
marine3.hp

In [6]:
def get_health(army):
    """
    Input is list of Units in an army
    Returns the sum of the remaining units in that army
    """
    health = 0
    for unit in army:
        health += unit.hp
    return health

In [7]:
def build_army(army_comp):
    """
    Input is a dictionary representing an army composition
    Returns a list of Units matching the army composition
    """
    army = []
    for name in army_comp:
        count = army_comp[name]
        while count >= 1:
            army.append(Unit(name))
            count -= 1
    return army

In [79]:
def get_attackable_unit(unit, enemy_army):
    """
    Input is attacking Unit and list of Units in enemy army
    Returns a random Unit in enemy army that attacking unit can attack
    If no enemy Unit can be attacked, return None
    """
    # create list of enemies unit can attack
    # targeting type (ground/air) and if enemy has hp>0
    enemies = []
    for enemy in enemy_army:
        if enemy.hp > 0:
            targetable = 0
            for target_type in unit.targetable:
                for enemy_type in enemy.type:
                    targetable += int(enemy_type == target_type)
            if targetable > 0:
                enemies.append(enemy)
    if len(enemies) == 0:
        return None
    else:
        # randomly chooses an enemy to attack
        index = random.randint(0,len(enemies)-1)
        enemy = enemies[index]
        return enemy

In [9]:
def deal_damage(army, enemy_army):
    """
    Input army is a list of attacking Units,
    enemy_army is list of Units being attacked.
    Calculates the damage dealt to enemy_army by all attacking units
    Updates the list of enemy Units with damaged health numbers
    """
    for unit in army:
        deal_unit_dps(unit, enemy_army)

In [10]:
def deal_unit_dps(unit, enemy_army):
    """
    Input is attacking Unit and list of Units being attacked
    Calculates the damage dealt to enemy army by the single unit
    Updates the list of enemy Units with damaged health numbers
    """
    damage = unit.dps
    bonus_dmg = unit.bonus_dps
    while (damage > 0) and (get_attackable_unit(unit, enemy_army) is not None):
        enemy = get_attackable_unit(unit, enemy_army)
        # check for bonus damage. bonus damage is kept seperate from
        # normal damage
        bonus = 0
        # so long as there is some bonus dps to damage, check if
        # there is at least one matching attribute
        if bonus_dmg > 0:
            for bonus_att in unit.bonuses:
                for att in enemy.attributes:
                    bonus += int(bonus_att == att)
        if bonus > 0:
            bonus_dmg_tmp = bonus_dmg
            bonus_dmg -= enemy.hp
            enemy.hp -= bonus_dmg_tmp
        dmg_temp = damage
        damage -= enemy.hp
        enemy.hp -= dmg_temp

In [70]:
def deal_ranged_dps(army, enemy_army):
    """
    Input army is a list of attacking Units,
    enemy_army is list of Units being attacked.
    Calculates the damage dealt to enemy_army only by ranged units
    Updates the list of enemy Units with damaged health numbers
    """
    for unit in army:
        if unit.ranged:
            deal_unit_dps(unit, enemy_army)

In [12]:
def remove_dead_units(army):
    """
    Input is a list of Units in an army
    Removes all Units with hp <=0 from that list
    Returns updated list
    """
    new_army = []
    for unit in army:
        if unit.hp >= 0:
            new_army.append(unit)
    return new_army

In [84]:
def combat_sim(army_comp1, army_comp2):
    """
    Input two army compositions written as dictionary with unit names as
    keys, unit counts as values. Simulates combat with the two army.
    Returns remaining army composition of army2
    """
    army1 = build_army(army_comp1)
    army2 = build_army(army_comp2)
    round1 = True
    while ((get_health(army1) > 0) and (get_health(army2) > 0)):
        if round1:
            deal_ranged_dps(army1, army2)
            deal_ranged_dps(army2, army1)
            army1 = remove_dead_units(army1)
            army2 = remove_dead_units(army2)
            round1 = False
        else:
            deal_damage(army1, army2)
            deal_damage(army2, army1)
            army1 = remove_dead_units(army1)
            army2 = remove_dead_units(army2)
    print("Attacking army:")
    print(army1)
    print()
    print("Enemy Army:")
    print(army2)

In [87]:
combat_sim(test_army_comp, enemy_army_comp)

Attacking army:
[]

Enemy Army:
[SCV_HP:35.2, Marine_HP:15.600000000000001, Marine_HP:31.200000000000003, Marine_HP:25.400000000000002]
